<a href="https://colab.research.google.com/github/meetarora10/veersa-hack/blob/main/model_assigning_doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install gtts
!pip install openai-whisper googletrans
!pip install deep_translator

In [3]:
from gtts import gTTS

# Hindi text
text = "मुझे बुखार है कल से"

# Convert to speech
tts = gTTS(text=text, lang='hi')
tts.save("mujhe_bukhar_hai_kal_se.mp3")
print("Audio saved as mujhe_bukhar_hai_kal_se.mp3")


Audio saved as mujhe_bukhar_hai_kal_se.mp3


In [4]:
import whisper
#import spacy
# 1. Load Whisper model
model = whisper.load_model('medium') # or "large" if you want
 # or "large" if you want

# 2. Transcribe audio file
result = model.transcribe('/content/mujhe_bukhar_hai_kal_se.mp3')
transcribed_text = result["text"]
print("Transcribed text:", transcribed_text)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text:  मुझे बुखार है कल से


In [5]:
from deep_translator import GoogleTranslator

translated_text = GoogleTranslator(source='auto', target='en').translate(transcribed_text)
print("Translated text:", translated_text)



Translated text: I have fever from tomorrow


In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Replace with your chosen model
model_name = "d4data/biomedical-ner-all"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

#text = "The patient is having stress and is depressed."

#entities = nlp(text)
# for ent in entities:
#     print(f"{ent['word']} ({ent['entity_group']}) - Score: {ent['score']:.2f}")


Device set to use cpu


In [8]:
# Now use this in your NER pipeline
entities = nlp(translated_text)
print(entities)

[{'entity_group': 'Sign_symptom', 'score': np.float32(0.9999182), 'word': 'fever', 'start': 7, 'end': 12}, {'entity_group': 'Time', 'score': np.float32(0.703359), 'word': 'tomorrow', 'start': 18, 'end': 26}]


In [9]:
def map_to_specialist(entities):
    specialist_map = {
        "Cardiologist": ["chest", "heart", "palpitation", "shortness of breath"],
        "Neurologist": ["headache", "seizure", "numbness", "dizziness"],
        "Gastroenterologist": ["stomach", "abdomen", "nausea", "diarrhea"],
        "Orthopedic": ["joint", "bone", "back", "fracture"],
        "Pulmonologist": ["cough", "lungs", "breathing", "asthma", "fever"],
        "Dermatologist": ["skin", "rash", "itching", "acne"],
        "Psychiatrist": ["depression", "anxiety", "sleep", "stress"],
    }

    # Prioritize based on entity_group and keywords
    for ent in entities:
        word = ent['word'].lower()
        label = ent['entity_group']

        if label in ["Sign_symptom", "Biological_structure"]:
            for specialist, keywords in specialist_map.items():
                for keyword in keywords:
                    if keyword in word:
                        return specialist

    return "General Physician"


In [10]:
map_to_specialist(entities)

'Pulmonologist'